### This example illustrates how to use mlflow to track parameters, metrics, artifacts and models while training a simple MNIST model.

In [ ]:
import mlflow.fastai
from fastai.vision.all import (
    CategoryBlock,
    DataBlock,
    GrandparentSplitter,
    ImageBlock,
    PILImage,
    URLs,
)
from fastai.vision.all import cnn_learner, get_image_files, parent_label, resnet18, untar_data

#### Splitting data between training and validation

In [ ]:
splitter = GrandparentSplitter(train_name="training", valid_name="testing")

# Prepare DataBlock which is a generic container to quickly build Datasets and DataLoaders
mnist = DataBlock(
    blocks=(ImageBlock(PILImage), CategoryBlock),
    get_items=get_image_files,
    splitter=splitter,
    get_y=parent_label,
)

#### Downloading MNIST data set and creating DataLoader from Datablock

In [ ]:
data = mnist.dataloaders(untar_data(URLs.MNIST), bs=256, num_workers=0)

##### Enabling autologging from mlflow

In [ ]:
mlflow.fastai.autolog()

##### Create Learner model

In [ ]:

learn = cnn_learner(data, resnet18)

##### Training model

In [ ]:

with mlflow.start_run() as run:
    # Train and fit with default or supplied command line arguments
    learn.fit_one_cycle(1, 0.1)
    mlflow.fastai.log_model(learn, "model")

#### Loading trained model from the run

In [ ]:
model_uri = "runs:/{}/model".format(run.info.run_id)
loaded_model = mlflow.fastai.load_model(model_uri)

##### Local prediction

In [1]:
data.train_ds[0][0]

NameError: name 'data' is not defined

In [112]:
loaded_model.predict(data.train_ds[0][0])

('3',
 TensorBase(3),
 TensorBase([1.3418e-13, 5.8279e-14, 5.2054e-12, 1.0000e+00, 3.8687e-13, 1.6166e-07,
         3.1506e-11, 4.8758e-13, 2.6322e-06, 4.4112e-10]))